In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')

df.head()

**WE need text and sentiment for analysis, hence clean the data**

In [ ]:
import re

df.text=df.text.apply(lambda x : x.lower())
df.text = df.text.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df.text.head()


In [ ]:

df.airline_sentiment=df.airline_sentiment.map({'neutral' : 1, 'positive':1,'negative':0})
df.airline_sentiment[0:10]


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
def cleanText(text):
    
    lemma = WordNetLemmatizer()
    stp = stopwords.words('english')
    
    # This means remove everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    
    text = text.lower()
    
    # This mean split sentences by words ("I am good" => ["I","am","good"])
    text = nltk.word_tokenize(text)
    
    # Lemmatizers convert words to their base form using dictionaries (going => go, bees => be , dog => dog)
    text = [lemma.lemmatize(word) for word in text]
    
    # We should remove stopwords, stopwords are the words that has no special meaning such as I,You,Me,Was
    text = [word for word in text if word not in stp]
    
    # Everything is ready, now we just need join the elements of lists (["feel","good"] => "feel good")
    text = " ".join(text)
    
    return text


In [ ]:
clean_text=[]
for w in df.text:
    clean_text.append(cleanText(w))
    

In [ ]:
x=np.array(clean_text)
y=df.airline_sentiment
y=np.array(y)

In [ ]:
print(x.shape,y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
#y = df[['pos_neg','pos_neu','neu_neg']]

#train_vectors = vectorizer.fit_transform(x).toarray()
X_train, X_test , Y_train, Y_test = train_test_split(x,y, test_size=0.2, random_state=50)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(train_vectors, Y_train)

In [ ]:
from  sklearn.metrics  import accuracy_score
predicted = clf.predict(test_vectors)
print(accuracy_score(Y_test,predicted))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Vocab_size= 500
oov_tok='<OOV>'

tokenizer=Tokenizer(num_words=Vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index

x_train_sequnces=tokenizer.texts_to_sequences(X_train)

tokenizer.fit_on_texts(X_test)
x_test_sequnces=tokenizer.texts_to_sequences(X_test)
x_test_sequnces[0]

In [ ]:
max_length = 100
trunc_type='post'
padding_type='post'
x_train_padded=pad_sequences(x_train_sequnces,maxlen=max_length,truncating=trunc_type,padding=padding_type)

x_test_padded=pad_sequences(x_test_sequnces,maxlen=max_length,truncating=trunc_type,padding=padding_type)


In [ ]:

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(1000,15,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')    
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(x_train_padded,Y_train,epochs=20,validation_data=(x_test_padded,Y_test))

In [ ]:
import matplotlib.pyplot as plt

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range=range(1,len(acc)+1)

plt.plot(epochs_range, acc, 'bo', label='Training Accuracy')
plt.plot(epochs_range, val_acc, 'b' ,label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs_range, loss,'bo', label='Training Loss')
plt.plot(epochs_range, val_loss,'b', label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

**Need to resduce over fitting**

In [ ]:
embedding_dim=128
model_multiple_bidi_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(3000, embedding_dim, input_length=x_train_padded.shape[1]),
    tf.keras.layers.SpatialDropout1D(0.4),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim,dropout=0.2,recurrent_dropout=0.2)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_multiple_bidi_lstm.summary()

In [ ]:
model_multiple_bidi_lstm.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])
history=model_multiple_bidi_lstm.fit(x_train_padded,Y_train,epochs=20,steps_per_epoch=100,validation_data=(x_test_padded,Y_test))

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range=range(1,len(acc)+1)

plt.plot(epochs_range, acc, 'bo', label='Training Accuracy')
plt.plot(epochs_range, val_acc, 'b' ,label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs_range, loss,'bo', label='Training Loss')
plt.plot(epochs_range, val_loss,'b', label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()